In [1]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash               
from pymongo import MongoClient
import dash

# Configure the necessary Python module imports
import dash_leaflet as dl
from dash import dcc, html, dash_table
import plotly.express as px
from dash.dependencies import Input, Output
import dash_core_components as dcc

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import base64

# URL Lib to make sure that our input is 'sane'
import urllib.parse

import AnimalShelter
from AnimalShelter import AnimalShelter


############################
# Data Manipulation / Model#
############################

shelter = AnimalShelter('aacuser', 'SNHU1234', 'nv-desktop-services.apporto.com', 30561, 'AAC', 'animals')


df = pd.DataFrame.from_records(shelter.read({}))


##########################
# Dashboard Layout / View#
##########################
app = JupyterDash('SimpleExample')

image_filename = 'Grazioso Salvare Logo.png'
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

app.layout = html.Div([
    html.Div(id='hidden-div', style={'display':'none'}),
    html.Center(html.B(html.H1('SNHU CS-340 Dashboard - Dylan Kusick'))),
    html.Hr(),
    html.Img(id='customer-image', src='data:image/png;base64,{}'.format(encoded_image.decode()),alt='customer image'),
    html.Div(
        dcc.RadioItems(
            id='filter-type',
            options=[
                {'label':'Water Rescue', 'value':'waterRescue'},
                {'label':'Mountain/Wilderness Rescue', 'value':'mountainRescue'},
                {'label':'Disaster Rescue and Individual Tracking', 'value':'disasterRescue'},
                {'label':'Reset', 'value':'reset'},
            ],
            value='reset'
        )
    ),
    html.Hr(),
    dash_table.DataTable(
        id='datatable-id',
        columns=[
            {"name":i, "id":i, "deletable":False, "selectable":True} for i in df.columns
        ],
        data=df.to_dict('records'),
        editable=False,
        filter_action="native",
        sort_action="native",
        row_selectable="single",
        selected_rows=[0],
        page_action="native",
        page_current=0,
        page_size=10
    ),
    html.Br(),
    html.Hr(),
    html.Div(className='row',
            style={'display':'flex'},
                children=[
        html.Div(
            id='graph-id',           #Histogram
            className='col s12 m6',
            ),
        html.Div(
            id='map-id',             #Map
            className='col s12 m6',
            )
        ])
    ])

##############################################
# Interaction Between Components / Controller#
##############################################

#Radio buttons
@app.callback([Output('datatable-id','data'),
               Output('datatable-id','columns')],
              [Input('filter-type', 'value')])

def update_dashboard(filter_type):
       
        if filter_type == 'waterRescue':                             #Water Rescue
            df = pd.DataFrame.from_records(shelter.read({
                "animal_type": "Dog",
                "breed": {"$in": ["Labrador Retriever Mix","Chesapeake Bay Retriever", "Newfoundland"]},
                "sex_upon_outcome": "Intact Female",
                "age_upon_outcome_in_weeks": {"$gte":26.0, "$lte":156.0}
            }))

        elif filter_type == 'mountainRescue':                        #Mountain or Wilderness Rescue
            df = pd.DataFrame.from_records(shelter.read({
                "animal_type": "Dog",
                "breed": {"$in": ["German Shepard","Alaskan Malamute","Old English Sheepdog", "Siberian Husky", "Rottweiler"]},
                "sex_upon_outcome": "Intact Male",
                "age_upon_outcome_in_weeks": {"$gte":26.0, "$lte":156.0}
            }))
            
        elif filter_type == 'disasterRescue':                        #Disaster Rescue or Individual Tracking
            df = pd.DataFrame.from_records(shelter.read({
                "animal_type": "Dog",
                "breed": {"$in": ["Doberman Pinscher","German Shepard","Golden Retriever", "Bloodhound","Rottweiler"]},
                "sex_upon_outcome": "Intact Male",
                "age_upon_outcome_in_weeks": {"$gte":20.0, "$lte":300.0}
            }))
            
        else:                                                        #Reset
            df = pd.DataFrame.from_records(shelter.read({}))
        
        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns]
        data=df.to_dict('records')
        
        
        return (data,columns)

#This callack will highlight a row on the data table when the user selects it
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]


#Chart
@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_viewport_data"),
    ])

def update_graphs(viewData):
    dff = pd.DataFrame.from_dict(viewData)
    return [
        dcc.Graph(            
            figure = px.histogram(dff, x='breed')
        )    
    ]


#Map
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', 'derived_virtual_selected_rows')])
def update_map(viewData, index):
    dff = pd.DataFrame.from_dict(viewData)
    # Because we only allow single row selection, the list can 
    # be converted to a row index here
    if index is None:
        row = 0
    else:
        row = index[0]
    
    # Austin TX is at [30.75, -97.48]
    
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'}, 
            center=[dff.iloc[row,13],dff.iloc[row,14]], zoom=10,
            children=[
            dl.TileLayer(id="base-layer-id"),
            # Marker with tool tip and popup
            # Column 13 and 14 define the grid-coordinates for
            # the map
            # Column 4 defines the breed for the animal
            # Column 9 defines the name of the animal 
            dl.Marker(position=[dff.iloc[row,13],dff.iloc[row,14]], 
                children=[
                dl.Tooltip(dff.iloc[row,4]),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[row,9])
                ])
            ])
        ])
    ]


app.run_server(debug=False)


/tmp/ipykernel_403537/3722825210.py:11: UserWarning: 
The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`
  import dash_core_components as dcc
 * Running on http://127.0.0.1:18541/ (Press CTRL+C to quit)
127.0.0.1 - - [11/Dec/2023 02:28:40] "GET /_alive_5c2fffdb-e591-4daf-8e95-628e3c301f22 HTTP/1.1" 200 -


Dash app running on http://127.0.0.1:18541/


127.0.0.1 - - [11/Dec/2023 02:28:41] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [11/Dec/2023 02:28:41] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [11/Dec/2023 02:28:41] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [11/Dec/2023 02:28:41] "GET /_dash-component-suites/dash/dash_table/async-highlight.js HTTP/1.1" 304 -
127.0.0.1 - - [11/Dec/2023 02:28:41] "GET /_dash-component-suites/dash/dash_table/async-table.js HTTP/1.1" 304 -
127.0.0.1 - - [11/Dec/2023 02:28:42] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [11/Dec/2023 02:28:43] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [11/Dec/2023 02:28:43] "POST /_dash-update-component HTTP/1.1" 500 -
127.0.0.1 - - [11/Dec/2023 02:28:45] "POST /_dash-update-component HTTP/1.1" 500 -
127.0.0.1 - - [11/Dec/2023 02:28:45] "POST /_dash-update-component HTTP/1.1" 500 -
127.0.0.1 - - [11/Dec/2023 02:28:45] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [11/Dec/2023 02:28:45] "GET /_dash-component-sui